In [19]:
import requests
import json

In [41]:
def reindex(analysisSettings={}, mappingSettings={}, movieDict={}):
    settings = {
        "number_of_ shards": 1,
        "index": {
            "analysis": analysisSettings,
        }
    }
    if mappingSettings:
        settings['mappings'] = mappingSettings
        
    resp = requests.delete('http://192.168.32.10:9200/tmdb')
    resp = requests.put('http://192.168.32.10:9200/tmdb', data=json.dumps(settings))
    bulkMovies = ""
    for id, movie in movieDict.iteritems():
        addCmd = {"index": {"_index": "tmdb",
                           "_type": "movie",
                           "_id": movie["id"]}}
        bulkMovies += json.dumps(addCmd) + "\n" + json.dumps(movie) + "\n"
    resp = requests.post("http://192.168.32.10:9200/_bulk", data=bulkMovies)

In [49]:
def extract():
    f = open('tmdb.json')
    if f:
        return json.loads(f.read());

In [50]:
def search(query):
    url = 'http://192.168.32.10:9200/tmdb/movie/_search'
    httpResp = requests.get(url, data=json.dumps(query))
    searchHits = json.loads(httpResp.text)['hits']
    print "Num\tRelevance Score \t\tMovie Title"
    for idx, hit in enumerate(searchHits['hits']):
        print "%s\t%s\t\t%s" % (idx + 1, hit['_score'], hit['_source']['title'])

In [29]:
usersSearch = 'star trek patrick stewart william shatner'
query = {
    "query":{
        "query_string":{
            "query": usersSearch,
            "fields": ["title", "overview", "cast.name^10", "directors.name"],
        }
    },
    "size": 5,
    "explain": True
}


In [9]:
search(query)

Num	Relevance Score 		Movie Title
1	0.21982981		Star Trek: Nemesis
2	0.16617933		Star Trek: Generations
3	0.14886102		Star Trek IV: The Voyage Home
4	0.11131736		Star Trek
5	0.089053884		Star Trek: First Contact


In [24]:
search(query)

Num	Relevance Score 		Movie Title
1	1.3178725		Star Trek: Nemesis
2	1.1268497		Star Trek IV: The Voyage Home
3	0.8510847		Star Trek
4	0.68086785		Star Trek: First Contact
5	0.60959524		Star Trek: Generations


In [28]:
search(query)

Num	Relevance Score 		Movie Title
1	2.9295588		Star Trek V: The Final Frontier
2	2.4750872		Star Trek: Generations
3	2.448519		Star Trek: Nemesis
4	1.557524		Star Trek: First Contact
5	1.3621734		Star Trek: Insurrection


In [30]:
search(query)

Num	Relevance Score 		Movie Title
1	1.1397538		Star Trek: Generations
2	0.6013422		Star Trek: Nemesis
3	0.58411956		Star Trek V: The Final Frontier
4	0.53611517		Star Trek II: The Wrath of Khan
5	0.48865876		Star Trek: The Motion Picture


### Mapping for custom all field

In [85]:
mappingSettings = {
    "movie": {
        "properties": {
            "people": {
                "properties": {
                    "name": {
                        "type": "string",
                        "analyzer": "english",
                        "fields": {
                            "bigrammed": {
                                "type": "string",
                                "analyzer": "englsh_bigrams"
                                }}}}},
            "cast": {
                "properties": {
                    "name": {
                        "type": "string",
                        "analyzer": "english",
                        "copy_to": "people.name",
                        "fields": {
                            "bigrammed": {
                                "type": "string",
                                "analyzer": "english_bigrams"
                            }
                        }
                    }
                }
            }
        }
    }
}

In [86]:
movieDict = extract()
analysisSettings={}
reindex(analysisSettings, mappingSettings, movieDict)

In [89]:
usersSearch = 'patrick stewart'
query = {
    "query":{
        "match": {
            "_all": usersSearch,
        }     
    }
}
search(query)

Num	Relevance Score 		Movie Title
1	0.3702025		Panic Room
2	0.34714928		Star Trek: Insurrection
3	0.34714928		Excalibur
4	0.3369333		Vertigo
5	0.33260775		Gnomeo & Juliet
6	0.3189207		Conspiracy Theory
7	0.3189207		Star Trek: First Contact
8	0.30435795		Star Trek: Nemesis
9	0.30375564		The Wolverine
10	0.30375564		Save the Last Dance
